In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_19093/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_19093/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_25_7,0.989852,0.730523,0.711307,0.973555,0.042590,1.801994,1.032638,0.023559,0.416933,0.206375,1.005940,0.215160,136.312250,215.539179,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_24_2,0.988460,0.730304,0.708871,0.980583,0.048431,1.803458,1.041352,0.130471,0.431934,0.220070,1.006755,0.229439,136.055238,215.282167,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_26_2,0.990435,0.730173,0.711870,0.996716,0.040144,1.804333,1.030625,0.024641,0.407715,0.200359,1.005599,0.208889,136.430580,215.657508,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_23_7,0.988498,0.730133,0.708686,0.946761,0.048271,1.804601,1.042013,0.133413,0.450633,0.219707,1.006733,0.229060,136.061849,215.288778,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_25_3,0.989266,0.730050,0.710072,0.977538,0.045048,1.805160,1.037054,0.020011,0.411935,0.212246,1.006283,0.221282,136.200040,215.426969,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,model_1_5_3,0.905648,0.654333,0.667563,0.901763,0.395976,2.311476,1.189108,0.650566,0.527422,0.629266,1.055231,0.656055,131.852804,211.079733,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
134,model_1_5_0,0.901543,0.650874,0.665322,0.901154,0.413201,2.334605,1.197124,0.654598,0.543689,0.642807,1.057633,0.670173,131.767640,210.994569,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
135,model_1_4_7,0.896278,0.646587,0.662842,0.149063,0.435298,2.363271,1.205994,0.110738,0.559494,0.659771,1.060715,0.687859,131.663448,210.890377,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
136,model_1_4_3,0.888042,0.640249,0.659400,0.134929,0.469865,2.405660,1.218306,0.112577,0.579662,0.685467,1.065537,0.714649,131.510620,210.737549,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
